<a href="https://colab.research.google.com/github/pa-shk/context_dependency_resolution/blob/main/T5_base_fine_tuning_2rca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Комментарии Аня

Получается, наш средний диалог

1. Порядка 5 реплик,
2. Около 300 символов,
3. В нем много пунктуации,
4. Тематически про увлечения, склонности и преференции (то же, что в исходных данных)

Тогда эти средние данные надо отделить от исходных, проверить на адекватность и сделать датасетом отправным.

По другим датасетам подумать, как их использовать для аугментации.

In [1]:
!pip install transformers[torch] sacrebleu evaluate
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install rouge-metric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import json
import itertools

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from collections import Counter
from tqdm import tqdm

# 2RCa

## Data loading

In [3]:
!wget https://github.com/anna-a-m/dataset-2rca/raw/refs/heads/main/data/2rca_checked_version.json -q

In [4]:
with open('2rca_checked_version.json') as f:
    data = json.load(f)

## Data samples

In [5]:
data[0]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_64c12a9b',
 'Phrase': 'Что читаешь? Мне нравится классика. Я тоже люблю пообщаться.',
 'Rewrite': 'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.'}

In [6]:
data[1]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_5cd9dc06',
 'Phrase': 'Люблю животных, просто обожаю, как и свою работу. Я фантастику люблю.',
 'Rewrite': 'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.'}

In [7]:
data[2]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.',
  'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.',
  'А я выращиваю фиалки и веду здоровый и активный образ жизни.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_9d73f6c2',
 'Phrase': 'Ух ты, интересно.',
 'Rewrite': 'Ух ты, интересно, ты фиалки выращиваешь.'}

In [8]:
data[3]

{'History': ['Привет, расскажи о себе.',
  'Привет! Под вкусный кофеек настроение поболтать появилось.',
  'Что читаешь? Мне нравится читать классику. Я тоже люблю пообщаться.',
  'Люблю животных, просто обожаю, как и свою работу. Я фантастику читать люблю.',
  'А я выращиваю фиалки и веду здоровый и активный образ жизни.',
  'Ух ты, интересно, ты фиалки выращиваешь.'],
 'Dia_ID_hash': 'dia_628e79cf',
 'Utt_ID_hash': 'utt_2bfa24e4',
 'Phrase': 'Ты случайно не принц на белом коне? Я его очень жду.',
 'Rewrite': 'Ты случайно не принц на белом коне? Я принца на белом коне очень жду.'}

Saving sample of data for manual examination

In [9]:
history, phrase, rewrite = [], [], []

for sample in data:
    history.append(sample["History"])
    phrase.append(sample["Phrase"])
    rewrite.append(sample["Rewrite"])

In [10]:
df = pd.DataFrame({"history": history, "phrase": phrase, "rewrite": rewrite})

## HF datasets

In [11]:
from datasets import DatasetDict, Dataset

ds = Dataset.from_pandas(df)
ds = ds.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_ds, test_ds = ds["train"], ds["test"]
test_ds = test_ds.train_test_split(test_size=0.5, shuffle=True, seed=42)
val_ds, test_ds = test_ds["train"], test_ds["train"]

ds["train"] = train_ds
ds["val"] = val_ds
ds["test"] = test_ds

In [12]:
ds

DatasetDict({
    train: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 4411
    })
    test: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 551
    })
    val: Dataset({
        features: ['history', 'phrase', 'rewrite'],
        num_rows: 551
    })
})

In [13]:
ds['train'][0]

{'history': ['Привет.',
  'Привет.',
  'Как твоё настроение сегодня?',
  'Настроение отлично, скоро куплю машину, а твоё настроение как?',
  'Здорово, ты скоро купишь машину! У меня тоже настроение на позитиве! Я очень давно хочу в Бразилию, и, кажется, сегодня моя мечта сбудется! Вечером поедем с подругой в турфирму оплачивать наш тур в Бразилию. А у тебя есть мечта?',
  'Давно хочу завести лабрадора, прям с детства хочу собаку. Может, когда дочка подрастет заведу себе лабрадора.',
  'Моей собаке уже 5 лет, и я даже не представляю, как я могла жить без своей собаки раньше?! Я думаю, что у тебя всё получится и у вас скоро обязательно появится питомец! Ведь собаки такие милые! Что сегодня будешь готовить на ужин?'],
 'phrase': 'Сегодня будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.',
 'rewrite': 'Сегодня на ужин будет мясо с кровью! Вот только надо в магазин... Эх, пойду прогуляюсь под дождём, это успокаивает.'}

# Model training

## Model loading

In [14]:
import random
import os
import numpy as np
import torch


def seed_everything(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True


seed_everything(42)

In [15]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

MODEL_NAME = 'ai-forever/ruT5-base'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [16]:
def preprocess_function(sample):
    inputs = [f"{context[-1]}</s>{phrase}</s>"
              for context, phrase in zip(sample["history"], sample["phrase"])]

    model_inputs = tokenizer(inputs, text_target=sample["rewrite"], max_length=128, truncation=True)
    return model_inputs

In [17]:
from transformers import DataCollatorForSeq2Seq


tokenized_gapping_ds = ds.map(preprocess_function, batched=True, batch_size=4)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model=MODEL_NAME,
                                       pad_to_multiple_of=8)

Map:   0%|          | 0/4411 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/t5/tokenization_t5.py:289: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


Map:   0%|          | 0/551 [00:00<?, ? examples/s]

Map:   0%|          | 0/551 [00:00<?, ? examples/s]

In [18]:
tokenized_gapping_ds

DatasetDict({
    train: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4411
    })
    test: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 551
    })
    val: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 551
    })
})

## Training

In [19]:
!sudo apt-get update
!sudo apt-get install libstdc++6

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,607 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,704 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,154 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:13 http://archive.ubuntu.c

In [20]:
# from peft import LoraConfig, get_peft_model
# from peft.utils.peft_types import TaskType

# config = LoraConfig(
#     r=2,
#     lora_alpha=2,
#     lora_dropout=0.2,
#     target_modules=["q", "v"],
#     bias="none",
#     task_type=TaskType.SEQ_2_SEQ_LM
# )


# # {
# #   "r": 4,
# #   "alpha": 8,
# #   "dropout": 0.2,
# #   "target_modules": ["q_proj"],
# #   "bias": "none",
# #   "task_type": "SEQ_CLS"
# # }

# peft_model = get_peft_model(model, config)

In [21]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [22]:
training_args = Seq2SeqTrainingArguments(
    overwrite_output_dir=True,
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=50,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
    logging_steps=10,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_gapping_ds["train"],
    eval_dataset=tokenized_gapping_ds["val"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

<ipython-input-22-75cf926a065a>:17: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: pvlshkunov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,5.409800,0.765807
2,0.939800,0.572155
3,0.667000,0.525874
4,0.547900,0.507955
5,0.426700,0.489329
6,0.405500,0.486064
7,0.355000,0.490348
8,0.310200,0.511001


KeyboardInterrupt: 

# Evaluation

In [23]:
import torch
from tqdm.notebook import tqdm

In [24]:
raw_test_results_new = []
model = model.to(DEVICE)

for i, encoding in tqdm(list(enumerate(tokenized_gapping_ds['test']['input_ids']))):
    input_ids=torch.tensor([encoding]).to(DEVICE)
    out = model.generate(inputs=input_ids,
                         eos_token_id=tokenizer.eos_token_id,
                         max_length=50,
                         num_beams=2,
                         early_stopping=True)
    out = tokenizer.decode(out[0][1:],
                           skip_special_tokens=True,
                           clean_up_tokenization_spaces=True)
    raw_test_results_new.append(out)

  0%|          | 0/551 [00:00<?, ?it/s]

In [25]:
tokenized_gapping_ds

DatasetDict({
    train: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4411
    })
    test: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 551
    })
    val: Dataset({
        features: ['history', 'phrase', 'rewrite', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 551
    })
})

In [26]:
import pandas as pd

raw_test_results = pd.DataFrame(raw_test_results_new, columns=['model_out_raw'])

raw_test_results['history'] = tokenized_gapping_ds['test']['history']
raw_test_results['text'] = tokenized_gapping_ds['test']['phrase']
raw_test_results['restored_text'] = tokenized_gapping_ds['test']['rewrite']

raw_test_results.head(10)

,model_out_raw,history,text,restored_text
0,"Я из Твери, а ты откуда?","[Привет., Привет. Откуда ты?]","Я из Твери, а ты?","Я из Твери, а ты откуда?"
1,"Да, именно Питерский зоопарк помню. Жаль, что ...","[Привет., Привет! Сколько тебе лет? Ты где-то ...","Да, именно Питерский. Жаль, что не в месте учи...","Да, именно Питерский политех. Жаль, что не вме..."
2,Мои дети - мои песики. Пока не нашла достойног...,"[Приветики! Как дела?, Привет! Всё хорошо, а у...",Мои дети - мои песики. Пока не нашла достойног...,Мои дети - мои песики. Пока не нашла достойног...
3,Где живёшь? В городе где-то живешь? Какие фрук...,"[Привет., Привет., Расскажи о себе., Я собираю...",Где живёшь? В городе где-то? Какие любишь?,Где живёшь? В городе где-то? Какие фрукты любишь?
4,"Да, это главное, с кем ты работаешь?","[Здравствуй. Как тебе погодка?, Привет, я дума...","Да, это самое главное, с кем ты работаешь?","Да, это самое главное что весна придет через п..."
5,"Иногда беру путевую с собой, у тебя кто?","[Привет, ты откуда? Я фотограф. Люблю путешест...","Иногда беру ее с собой, у тебя кто?","Иногда беру дочь с собой в путешествие, а у те..."
6,Я всегда за новые знакомства.,"[Добрый вечер, вы не против новых знакомств?]",Я всегда за.,Я всегда за знакомства.
7,Я пью евицу. А вы чем занимаетесь?,"[Привет., Привет, Алла., Чем вы занимаетесь?]",Певица. А вы?,Я певица. А вы чем занимаетесь?
8,Чтобы не испортить зрение и осанку за компьюте...,"[Привет., О, привет! Рад новому знакомству! Ты...",Чтобы не испортить зрение и осанку за компьюте...,Чтобы не испортить зрение и осанку за компьюте...
9,"Да, и учусь, и работаю.","[Привет., Привет. Как дела?, Отлично у меня де...","Да, и учусь, и работаю.","Да, я и учусь, и работаю."


In [29]:
raw_test_results.to_csv(f"{MODEL_NAME.split('/')[-1]}_raw_test_results.csv")

In [30]:
import numpy as np
import evaluate
from rouge_metric import PyRouge

bleu = evaluate.load("sacrebleu")
rouge = PyRouge(rouge_n=(4), skip_gap=4)


class RestorationFScore:

    def __init__(self, tokenizer, n_gram: int=2):
        self.n_gram = n_gram
        self.tokenizer = tokenizer

    def preprocess(self, sents):
        for sent in sents:
            sent_tokenize = self.tokenizer(sent)['input_ids']
            yield [tuple(sent_tokenize[i:i+self.n_gram]) for i, _ in enumerate(sent_tokenize)]

    def _itereval(self):
        for i, predictions in enumerate(self.predictions):
            restored_ngrams = set(predictions).difference(self.references[i])
            ngrams_in_ref = set(self.rewrites[i]).difference(self.references[i])
            interagree = ngrams_in_ref.intersection(restored_ngrams)
            if len(restored_ngrams):
                precision = len(interagree) / len(restored_ngrams)
            else:
                precision = 0.
            if len(ngrams_in_ref):
                recall = len(interagree) / len(ngrams_in_ref)
            else:
                recall = 0.
            if precision or recall:
                yield 2 * ((precision * recall) / (precision + recall))
            else:
                yield 0.

    def evaluate(self, predictions: list,
                 references: list, rewrites: list):
        self.predictions = [p for p in self.preprocess(predictions)]
        self.references = [p for p in self.preprocess(references)]
        self.rewrites = [p for p in self.preprocess(rewrites)]
        return np.mean(list(self._itereval()))

In [31]:
def callculate_metrics(row):
    row["bleu_score"] = bleu.compute(predictions=row.model_out_raw,
                                     references=row.text)['score']

    rouge_scores = rouge.evaluate(row.model_out_raw,
                                  [[t] for t in row.text])
    for k in rouge_scores:
        row[k] = rouge_scores[k]['f']

    for n in range(1, 5):
        rf_score = RestorationFScore(tokenizer, n)
        row[f"rf_score_{n}"] = rf_score.evaluate(predictions=row.model_out_raw,
                                                 references=row.text,
                                                 rewrites=row.restored_text)
    return row

In [32]:
raw_test_results["type"] = "2rca"
raw_test_results = raw_test_results.groupby(by="type").agg(list)

In [33]:
raw_test_results.apply(callculate_metrics, axis=1).drop(columns=["model_out_raw", "history", "text", "restored_text"])

,bleu_score,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rf_score_1,rf_score_2,rf_score_3,rf_score_4
type,,,,,,,,,,
2rca,73.343847,0.734728,0.618773,0.530793,0.46508,0.734235,0.40779,0.324883,0.289028,0.270044
